In [1]:
import sys
sys.path.append("..")
import json
import random
import dadagp
import os
from model_ead import TransformerXL
import make_loops as loops
import guitarpro
import pickle
import torch
import yaml

d:\Documents\Queen Mary\dev\msc_thesis\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
FILELIST_PATH = "D:\\Documents\\DATA\\DadaGP-4-only-lps-3-dens-per-inst\\file_list_loops.json"
ROOT_PATH = "D:\\Documents\\DATA\\DadaGP-4-only-lps-3-dens-per-inst"
OUTPUT_PATH = "D:\\Documents\\DATA\\DadaGP-study-samples-test-params2" #1 is 1.3/0.9, 2 is 1./0.9, 3 is 1.2/0.95 4 is 1.2/0.8
NUM_SAMPLES = 20

In [3]:
t_params = [0.8,0.9,1.0,1.1,1.25,1.5]
p_params = [0.85,0.9,0.95]

In [4]:
os.makedirs(os.path.join(OUTPUT_PATH, "human"))

In [5]:
with open(FILELIST_PATH, "r") as f:
    file_list =  json.load(f)
num_files = len(file_list)
print("{} files".format(num_files))

18987 files


In [6]:
sampled_idxes = random.sample(range(num_files), NUM_SAMPLES)

In [7]:
human_sample_data = {}
primers = []
for id, idx in enumerate(sampled_idxes):
    filepath = file_list[idx]
    file = os.path.join(ROOT_PATH, file_list[idx])
    #print(file)
    with open(file, "r") as f:
        text = f.read()
    list_words = text.split("\n")

    header_data = list_words[:4]
    main_data = list_words[4:]

    num_measures = 0
    measure_idx = []
    for i,token in enumerate(main_data):
        if token == "new_measure":
            num_measures += 1
            measure_idx.append(i)
    num_segments = int(num_measures / 4)
    measure_idx.append(len(main_data))

    rand_segment = random.randint(0, num_segments - 1)
    human_sample_data[id] = rand_segment, filepath
    measure_start = rand_segment * 4

    start_idx = measure_idx[measure_start]
    end_idx = measure_idx[measure_start + 4]

    final_loop = header_data + main_data[start_idx:end_idx]
    if final_loop[-1] != "end":
        final_loop.append("end")
    primer = []
    for token in final_loop:
        if "artist:" in token:
            primer.append("artist:unknown_artist")
        else:
            primer.append(token)
        if "wait:" in token:
            break
    primers.append(primer)
    #print(primer)

    token_path = os.path.join(OUTPUT_PATH, "human", "ex_" + str(id) + ".txt")
    dadagp_path = os.path.join(OUTPUT_PATH, "human", "ex_" + str(id) + ".gp5")
    file_out = open(token_path, "w")
    file_out.write("\n".join(final_loop))
    file_out.close()
    dadagp.dadagp_decode(token_path, dadagp_path)

path_json = os.path.join(OUTPUT_PATH, "sampled_loops_info.json")
with open(path_json, 'w') as f:
    json.dump(human_sample_data, f)


In [8]:
cfg = yaml.full_load(open("../full-data-config_5_lat1024.yml", 'r')) 
inferenceConfig = cfg['INFERENCE']

os.environ['CUDA_VISIBLE_DEVICES'] = inferenceConfig['gpuID']

CHECKPOINT_FOLDER = inferenceConfig['experiment_dir']

checkpoint_type = inferenceConfig['checkpoint_type']
if checkpoint_type == 'best_train':
        model_path = os.path.join(CHECKPOINT_FOLDER, 'model_best.pth.tar')
elif checkpoint_type == 'best_val':
        model_path = os.path.join(CHECKPOINT_FOLDER, 'model_best_val.pth.tar')
elif checkpoint_type == 'epoch_idx':
        model_path = os.path.join(CHECKPOINT_FOLDER, 'ep_{}.pth.tar'.format(str(inferenceConfig['model_epoch'])))

pretrainCfg = yaml.full_load(open(os.path.join("..", CHECKPOINT_FOLDER,"full-data-config.yml"), 'r')) 
modelConfig = pretrainCfg['MODEL']

event2word = pickle.load(open(os.path.join("..", inferenceConfig['vocab_data_path']), 'rb'))
word2event = pickle.load(open(os.path.join("..", inferenceConfig['rev_vocab_data_path']), 'rb'))

# declare model
device = torch.device("cuda" if not inferenceConfig["no_cuda"] and torch.cuda.is_available() else "cpu")
print('Device to generate:', device)

# declare model
model =  TransformerXL(
        modelConfig,
        inferenceConfig['gpuID'],
        event2word=event2word, 
        word2event=word2event, 
        is_training=False)

Device to generate: cuda
cuda:0


In [9]:
for T in t_params:
    for P in p_params:
        os.makedirs(os.path.join(OUTPUT_PATH, "machine_"+str(T).replace(".","")+"_"+str(P).replace(".","")))

In [10]:
NUM_BARS = 16
LOOP_SIZE = 4
MIN_LEN = 4
MIN_REP_BEATS = 2.0
for T in t_params:
    for P in p_params:
        for idx, primer in enumerate(primers):
            print(idx, primer)
            loops_length = 0
            while loops_length < 10:
                generated = model.inference_single_from_primer(os.path.join("..",model_path), ['temperature', 'nucleus'], {'t': T ,'p': P, 'num_bars': NUM_BARS}, primer)
                song = dadagp.tokens2guitarpro(generated, verbose=False)
                # Appears at the top of the GP score
                song.artist = generated[0]
                song.album = 'Generated by DadaGP'
                song.title = "untitled"
                dadagp_path = os.path.join(OUTPUT_PATH, "machine_"+str(T).replace(".","")+"_"+str(P).replace(".",""), "ex_" + str(idx) + "_full" + ".gp5")
                guitarpro.write(song, dadagp_path) # GP file transcoded into tokens and back again

                track_list, time_signatures = loops.create_track_list(song)
                beats_per_bar = 4
                min_beats = beats_per_bar * LOOP_SIZE
                max_beats = beats_per_bar * LOOP_SIZE
                lead_mat, lead_dur, melody_seq = loops.calc_correlation(track_list, 0) #assuming first instrument is most loopable
                _, loop_endpoints = loops.get_valid_loops(melody_seq, lead_mat, lead_dur, min_len=MIN_LEN, min_beats=min_beats, max_beats=max_beats, min_rep_beats=MIN_REP_BEATS)
                token_list = generated[0:4]
                if len(loop_endpoints) > 0:
                    token_list = loops.unify_loops(generated, loop_endpoints, density=4)
                token_list_repeats = loops.get_repeats(generated, density=4)
                token_list = token_list + token_list_repeats
                if token_list[-1] != "end":
                    token_list.append("end")
                loops_length = len(token_list)
                if loops_length < 10:
                    print("REGENERATING, NO LOOPS FOUND")
                    continue
                print(len(token_list))
                song = dadagp.tokens2guitarpro(token_list, verbose=False)
                # Appears at the top of the GP score
                song.artist = generated[0]
                song.album = 'Generated by DadaGP'
                song.title = "untitled"
                dadagp_path = os.path.join(OUTPUT_PATH, "machine_"+str(T).replace(".","")+"_"+str(P).replace(".",""), "ex_" + str(idx) + "_loops" + ".gp5")
                guitarpro.write(song, dadagp_path) # GP file transcoded into tokens and back again


                header_data = token_list[:4]
                main_data = token_list[4:]

                num_measures = 0
                measure_idx = []
                for i,token in enumerate(main_data):
                    if token == "new_measure":
                        num_measures += 1
                        measure_idx.append(i)
                num_segments = int(num_measures / 4)
                measure_idx.append(len(main_data))

                rand_segment = random.randint(0, num_segments - 1)
                human_sample_data[id] = rand_segment, filepath
                measure_start = rand_segment * 4

                start_idx = measure_idx[measure_start]
                end_idx = measure_idx[measure_start + 4]

                final_loop = header_data + main_data[start_idx:end_idx]
                if final_loop[-1] != "end":
                    final_loop.append("end")

                token_path = os.path.join(OUTPUT_PATH, "machine_"+str(T).replace(".","")+"_"+str(P).replace(".",""), "ex_" + str(idx) + ".txt")
                dadagp_path = os.path.join(OUTPUT_PATH, "machine_"+str(T).replace(".","")+"_"+str(P).replace(".",""), "ex_" + str(idx) + ".gp5")
                file_out = open(token_path, "w")
                file_out.write("\n".join(final_loop))
                file_out.close()
                dadagp.dadagp_decode(token_path, dadagp_path)
    

0 ['artist:unknown_artist', 'downtune:0', 'tempo:130', 'start', 'new_measure', 'measure:repeat_open', 'distorted1:note:s4:f8', 'distorted1:note:s5:f8', 'distorted1:note:s6:f6', 'drums:note:49', 'drums:note:40', 'wait:960']
Pretrained model config for cuda:0: epoch 50 best_loss 0.1332303942674966
..\./outputs/5_lat102420220714-235039\ep_50.pth.tar loaded on cuda:0
960
['distorted1', 'drums']
Primer: ['artist:unknown_artist', 'downtune:0', 'tempo:130', 'start', 'new_measure', 'measure:repeat_open', 'distorted1:note:s4:f8', 'distorted1:note:s5:f8', 'distorted1:note:s6:f6', 'drums:note:49', 'drums:note:40', 'wait:960']
455
1 ['artist:unknown_artist', 'downtune:0', 'tempo:80', 'start', 'new_measure', 'measure:repeat_open', 'distorted0:note:s3:f7', 'distorted0:note:s4:f7', 'distorted0:note:s5:f5', 'distorted0:note:s6:f-2', 'distorted1:rest', 'bass:note:s5:f-2', 'drums:note:49', 'wait:480']
Pretrained model config for cuda:0: epoch 50 best_loss 0.1332303942674966
..\./outputs/5_lat10242022071